# Practice notebook for confidence intervals using NHANES data

This notebook will give you the opportunity to practice working with confidence intervals using the NHANES data.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [145]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

da = pd.read_csv("nhanes_2015_2016.csv")
print(da['RIDAGEYR'])

0       62
1       53
2       78
3       56
4       42
        ..
5730    76
5731    26
5732    80
5733    35
5734    24
Name: RIDAGEYR, Length: 5735, dtype: int64


## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

In [8]:
df = da[(da['RIDAGEYR'] >= 35) & (da['RIDAGEYR'] <= 50) & (da['RIAGENDR'] == 2)]

df['DMDMARTL'] = df.DMDMARTL.replace({1:'Married',2:'Not_Married',3:'Not_Married', 4:'Not_Married', 5:'Not_Married', 6:'Not_Married', 77: np.nan, 99: np.nan})
df['DMDEDUC2'] = df.DMDEDUC2.replace({7: np.nan, 9: np.nan})
df['DMDEDUC2'] = df.DMDEDUC2[df['DMDEDUC2'] >= 4]
df['DMDEDUC2'] = df.DMDEDUC2.replace({4: "Some College", 5: "College Graduate"})
df = df[["RIAGENDR","DMDMARTL","DMDEDUC2"]].dropna()

#rint(pd.crosstab(df.DMDMARTL, df.DMDEDUC2))

dx = df.groupby(df.DMDMARTL).agg({"DMDEDUC2": [lambda x: np.mean(x == "College Graduate"), np.size]})
dx.columns = ["Proportion", "Total_n"]
print(dx)

p = dx.Proportion.Married
n = dx.Total_n.Married

se_Married = np.sqrt(p * (1 - p) / n)
print("Se Married:", se_Married)

p = dx.Proportion.Not_Married
n = dx.Total_n.Not_Married

se_Not_Married = np.sqrt(p * (1 - p) / n)
print("Se Not Married:",se_Not_Married)

p = dx.Proportion.Married 
n = dx.Total_n.Married 
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print("Lcb Married: ",lcb,"Ucb Married: ", ucb)

p = dx.Proportion.Not_Married 
n = dx.Total_n.Not_Married 
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print("Lcb Not Married: ",lcb,"Ucb Not Married: ", ucb)



             Proportion  Total_n
DMDMARTL                        
Married        0.538206      301
Not_Married    0.378947      190
Se Married: 0.02873526218165375
Se Not Married: 0.03519466619186639


__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case. 

By sampling the data we can clearly see that the biggest percentage of woman graduated are married 

__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

## Question 2

Construct 95% confidence intervals for the proportion of smokers who are female, and for the proportion of smokers who are male.  Then construct a 95% confidence interval for the difference between these proportions.

In [42]:
da.SMQ020 = da.SMQ020.replace({1: "Smoker", 2:"Not SMoker", 7: np.nan, 9:np.nan})
da.RIAGENDR = da.RIAGENDR.replace({1:"Male", 2:"Female"})

df = da[["SMQ020","RIAGENDR"]].dropna()
dx = df.groupby(df.RIAGENDR).agg({"SMQ020": [lambda x: np.mean(x=="Smoker"), np.size]})

dx.columns = ['Proportion','Total_n']
print(dx)

p = dx.Proportion.Female
n = dx.Total_n.Female

se_female = np.sqrt(p * (1-p) / n)
print("Se Female:", se_female)

p = dx.Proportion.Male
n = dx.Total_n.Male 

se_male = np.sqrt(p * (1-p) / n)
print("Se Male:", se_male)

p = dx.Proportion.Female 
n = dx.Total_n.Female 
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print("Lcb Female: ",lcb,"Ucb Female: ", ucb)


p = dx.Proportion.Male 
n = dx.Total_n.Male 
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print("Lcb Male: ",lcb,"Ucb Male: ", ucb)

          Proportion  Total_n
RIAGENDR                     
Female      0.304845     2972
Male        0.513258     2753
Se Female: 0.008444152146214435
Se Male: 0.009526078653689868
Lcb Female:  0.288294683866098 Ucb Female:  0.32139576027925865
Lcb Male:  0.49458714955108174 Ucb Male:  0.531929377873546


__Q2a.__ Discuss why it may be relevant to report the proportions of smokers who are female and male, and contrast this to reporting the proportions of males and females who smoke.

__Q2b.__ How does the width of the confidence interval for the difference of the two proportions compare to the widths of the confidence intervals for each proportion separately?

## Question 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters   

In [39]:
dx = pd.DataFrame(index = da.index,columns=["BMXHTinch", "BMXHTcm"])
dx['BMXHTcm'] = da.BMXHT.values
dx['BMXHTcm'] = da.BMXHT.apply(lambda x:  np.nan if x == 0 else x  )
dx['BMXHTinch'] = da.BMXHT.apply(lambda x: x/2.54)
dx = dx.dropna()
Se_cm = np.std(dx.BMXHTcm)/np.sqrt(len(dx))
mean = np.mean(dx.BMXHTcm)
lcb_height_cm = mean - 1.96 * Se_cm
ucb_height_cm = mean + 1.96 * Se_cm

print("Lcb_Height_cm: ", lcb_height_cm, "Ucb_Height_cm: ", ucb_height_cm)

Se_inch = np.std(dx.BMXHTinch)/np.sqrt(len(dx))
mean = np.mean(dx.BMXHTinch)
lcb_height_inch = mean - 1.96 * Se_inch
ucb_height_inch = mean + 1.96 * Se_inch

print("Lcb_Height_inch: ", lcb_height_inch, "Ucb_Height_inch: ", ucb_height_inch)

from_inch_to_cm_lcb = lcb_height_inch * 2.54 
from_inch_to_cm_ucb = ucb_height_inch * 2.54
print("From inches to cm, Lcb: ", from_inch_to_cm_lcb, "Ucb :,", from_inch_to_cm_ucb)

Lcb_Height_cm:  165.8805695573686 Ucb_Height_cm:  166.40509940085403
Lcb_Height_inch:  65.30731084935765 Ucb_Height_inch:  65.51381866175349
From inches to cm, Lcb:  165.88056955736843 Ucb :, 166.40509940085386


__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

## Question 4

Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [76]:
da["Ages_range"] = pd.cut(da.RIDAGEYR, [18,28,38,48,58,68,78])
dz = da.groupby(["Ages_range", "RIAGENDR"]).agg({"BMXBMI": [np.size, np.mean, np.std]}).unstack()
dz["BMXBMI", "Sem", "Female"] = dz["BMXBMI", "std", "Female"] / np.sqrt(dz["BMXBMI", "size", "Female"])
dz["BMXBMI", "Sem", "Male"] = dz["BMXBMI", "std", "Male"] / np.sqrt(dz["BMXBMI", "size", "Male"])

df_results = pd.DataFrame(columns=['difference_mean', 'difference_sem','difference_lcb','difference_ucb','difference_width'])
df_results.difference_mean= dz["BMXBMI", "mean", "Male"] - dz["BMXBMI", "mean", "Female"]
df_results.difference_sem = np.sqrt(dz.BMXBMI.Sem.Female**2 + dz.BMXBMI.Sem.Male**2)
df_results.difference_lcb = df_results.difference_mean - 1.96 * df_results.difference_sem
df_results.difference_ucb = df_results.difference_mean + 1.96 * df_results.difference_sem
df_results.difference_width = df_results.difference_lcb - df_results.difference_ucb
df_results



,difference_mean,difference_sem,difference_lcb,difference_ucb,difference_width
Ages_range,,,,,
"(18, 28]",-0.961247,0.476973,-1.896114,-0.026381,-1.869734
"(28, 38]",-0.246263,0.474473,-1.176230,0.683705,-1.859935
"(38, 48]",-1.489086,0.468563,-2.407470,-0.570703,-1.836767
"(48, 58]",-1.402230,0.468120,-2.319745,-0.484714,-1.835031
"(58, 68]",-1.822202,0.453239,-2.710551,-0.933853,-1.776698
"(68, 78]",-1.817548,0.507502,-2.812252,-0.822844,-1.989407


__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial diferences in the confidence interval widths that you see.

We can notice that for people aged over 58 the difference on BMI is greater for older people between male and female

## Question 5

Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

In [82]:
systolic_1 = da['BPXSY1']
systolic_2 = da['BPXSY2']
mean_systolic_1 = np.mean(systolic_1)
mean_systolic_2 = np.mean(systolic_2)

se_systolic_1 = np.std(systolic_1)/np.sqrt(len(systolic_1))
se_systolic_2 = np.std(systolic_2)/np.sqrt(len(systolic_2))

lcb_systolic_1 = mean_systolic_1 - 1.96 * se_systolic_1
lcb_systolic_2 = mean_systolic_2 - 1.96 * se_systolic_2

ucb_systolic_1 = mean_systolic_1 + 1.96 * se_systolic_1
ucb_systolic_2 = mean_systolic_2 + 1.96 * se_systolic_2

print("Interval of confidence for Systolic Pressure 1: ", lcb_systolic_1, ucb_systolic_1 )
print("Interval of confidence for Systolic Pressure 2: ", lcb_systolic_2, ucb_systolic_2 )

difference_se = se_systolic_1 - se_systolic_2
difference_mean = mean_systolic_1 - mean_systolic_2
difference_lcb  = difference_mean - 1.96 * difference_se
difference_ucb = difference_mean + 1.96 * difference_se

print("The interval of confidence between the 2 pressure are: ", difference_lcb, difference_ucb)

Interval of confidence for Systolic Pressure 1:  124.60634562793352 125.5628822928219
Interval of confidence for Systolic Pressure 2:  124.30355371876219 125.26248060824774
The interval of confidence between the 2 pressure are:  0.30279190917132703 0.3004016845741605


__Q5a.__ Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?

No, there's actually no difference between the two mesaurment


__Q5b.__ Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

## Question 6

Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

In [ ]:
da["SMQ020"] = da.SMQ020.replace({1: 'Smoker', 2: 'Not Smoker', 7: np.nan, 9: np.nan})
dz = da[["SMQ020", "RIDAGEYR"]].dropna()
smokers = dz[dz["SMQ020"] == "Smoker"]
not_smokers = dz[dz["SMQ020"] == "Not Smoker"]

se_smokers = np.std(smokers.RIDAGEYR)/np.sqrt(dz.SMQ020.size)
se_not_smokers = np.std(not_smokers.RIDAGEYR)/np.sqrt(dz.SMQ020.size)

lcb_difference = (np.mean(not_smokers.RIDAGEYR) - np.mean(smokers.RIDAGEYR)) - 1.96 * (se_not_smokers - se_smokers)
ucb_difference = (np.mean(not_smokers.RIDAGEYR) - np.mean(smokers.RIDAGEYR)) + 1.96 * (se_not_smokers - se_smokers)

print("Interval for the mean difference between the average age of a smoker: \n", lcb_difference, ucb_difference)
print("Age mean of non-smokers is: ", np.mean(smokers.RIDAGEYR))

__Q6a.__ Use graphical and numerical techniques to compare the variation in the ages of smokers to the variation in the ages of non-smokers.  

In [167]:
import matplotlib.pyplot as plt
da["Ages_range"] = pd.cut(da.RIDAGEYR, [18,28,38,48,58,68,78])
dx = da.groupby(["Ages_range"], as_index = False)["SMQ020"].count()
#fig, (ax1, ax2) = plt.subplots(nrows = 2, ncols = 2, figsize = (5, 3))
#ax.hist()
dx

__Q6b.__ Does it appear that uncertainty about the mean age of smokers, or uncertainty about the mean age of non-smokers contributed more to the uncertainty for the mean difference that we are focusing on here?